In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-04-28 03:26:30 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...


In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2




<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [ ]:
using Pkg

In [ ]:
Pkg.add(url="https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git")

In [ ]:
Pkg.add("QuantumControl")

In [ ]:
using GRAPELinesearchAnalysis

In [ ]:
using QuantumControl

In [ ]:
Pkg.add("QuantumControlBase")

In [ ]:
using QuantumControlBase

In [ ]:
ϵ1(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=300, t_rise=0.3, func=:blackman);
ϵ2(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=300, t_rise=0.3, func=:blackman);

In [ ]:
"""Two-level-system Hamiltonian."""
function hamiltonian(Ω1=1.0, ϵ1=ϵ1,ϵ2=ϵ2, Ω2=1.0, J=0.01)
    Iσ̂_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0  1  0
        0  0  0  -1
    ]
    Iσ̂_x = ComplexF64[
        0  1  0  0
        1  0  0  0
        0  0  0  1
        0  0  1  0
    ]
    σ̂_xI = ComplexF64[
        0  0  1  0
        0  0  0  1
        1  0  0  0
        0  1  0  0
    ]
    σ̂_zI = ComplexF64[
        1  0  0  0
        0  1  0  0
        0  0 -1  0
        0  0  0  -1
    ]
    σ̂_x_x = ComplexF64[
        0  0  0  1
        0  0  1  0
        0  1  0  0
        1  0  0  0
    ]
    σ̂_y_y = ComplexF64[
        0  0  0 -1
        0  0  1  0
        0  1  0  0
       -1  0  0  0
    ]
    σ̂_z_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0 -1  0
        0  0  0  1
    ]
    Ĥ₀ = -0.5 * (Ω1 * Iσ̂_z + Ω2 * σ̂_zI) + J * (σ̂_z_z)
    Ĥ₁ = Iσ̂_x
    Ĥ2 = σ̂_xI
    return (Ĥ₀, (Ĥ₁, ϵ1), (Ĥ2, ϵ2))
end;

H = hamiltonian();

In [ ]:
tlist = collect(range(0, 300, length=3000));

In [ ]:
function ket(label)
    result = Dict("00" => Vector{ComplexF64}([1, 0, 0, 0]), "01" => Vector{ComplexF64}([0, 1, 0, 0]), "10" => Vector{ComplexF64}([0, 0, 1, 0]), "11" => Vector{ComplexF64}([0, 0, 0, 1]))
    return result[string(label)]
end;

In [ ]:
basis = [ket("00"),ket("01"),ket("10"),ket("11")];

In [ ]:
function CRU(teta, phi, landa , gama)
    CU_gate= [[1, 0, 0, 0] [0, 1, 0, 0] [0, 0, exp(1im * landa) * cos(teta/2),  -1*exp(1im * (gama + landa)) * sin(teta/2)
] [0, 0, exp(1im *(landa + phi)) * sin(teta/2), exp(1im *(landa + phi + gama)) *cos(teta/2)]]

end

CRU (generic function with 1 method)

In [ ]:
RandU_2Q=Matrix{ComplexF64}[]
N=100
for i in 1:N
   teta=rand(0:pi)
   phi=rand(0:2*pi)
   landa=rand(0:2*pi)
   gama=rand(0:2*pi)
   push!(RandU_2Q, CRU(teta, phi, landa, gama))
end


In [ ]:
using LinearAlgebra

In [ ]:
basis_tgt=Array[]
for i in 1:100
   b= transpose(RandU_2Q[i]) * basis
   push!(basis_tgt, b)

end


In [ ]:
objectives=Array[]
for i in 1:100
   obj = [Objective(initial_state = Ψ, target_state = Ψtgt, generator = H) for
    (Ψ, Ψtgt) ∈ zip(basis, basis_tgt[i])];
    push!(objectives, obj)
end


In [ ]:
problem=[]
for i in 1:100

    prob = ControlProblem(
    objectives=objectives[i],
    tlist=tlist,
    pulse_options=Dict(),
    iter_stop=500,
    J_T=QuantumControl.Functionals.J_T_sm,
    check_convergence=res -> begin
        ((res.J_T < 1e-3) && (res.converged = true) && (res.message = "J_T < 10⁻³"))
    end,
    );
    push!(problem, prob)
end

In [ ]:
Pkg.add("QuantumPropagators")
using QuantumPropagators

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [7bf12567] + QuantumPropagators v0.5.0
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
function store_pulses(wrk, iteration, _...)
    L = length(wrk.controls)
    ϵ_opt = reshape(wrk.pulsevals, L, :)
    opt_pulses = [QuantumControl.Controls.discretize_on_midpoints(ϵ_opt[l, :], tlist) for l=1:L]
    return Tuple(opt_pulses)
end

store_pulses (generic function with 1 method)

In [ ]:
Pkg.add("GRAPE")
using GRAPE

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [6b52fcaf] + GRAPE v0.5.4
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
wrk=[]
for i in 1:100
  wrkk=GRAPE.GrapeWrk(problem[i]; )
  push!(wrk, wrkk)
end


In [ ]:
results=[]

for i in 1:100
  opt_result_LBFGSB = optimize(

      problem[i],
      method = :grape,
      force = true,
      info_hook = QuantumControlBase.chain_infohooks(

          QuantumControl.GRAPE.print_table,
          store_pulses
      )
  );
  push!(results, opt_result_LBFGSB)
end


 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   9.16e-01   6.35e-01        n/a    1(0)     8.7
     1   8.24e-01   5.02e-01  -9.14e-02    2(0)     1.6
     2   7.70e-01   1.19e+00  -5.44e-02    1(0)     0.4
     3   5.25e-01   1.94e+00  -2.45e-01    3(0)     1.3
     4   3.65e-01   2.18e+00  -1.60e-01    2(0)     0.8
     5   1.53e-01   1.43e+00  -2.13e-01    1(0)     0.5
     6   4.51e-02   5.46e-01  -1.08e-01    1(0)     0.4
     7   2.31e-02   3.63e-01  -2.20e-02    1(0)     0.5
     8   1.38e-02   2.06e-01  -9.31e-03    1(0)     0.4
     9   5.81e-03   1.60e-01  -7.99e-03    1(0)     0.5
    10   1.52e-03   1.08e-01  -4.29e-03    1(0)     0.4
    11   5.66e-04   7.86e-02  -9.55e-04    2(0)     2.7
 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   9.81e-01   4.89e-01        n/a    1(0)     0.8
     1   5.38e-01   1.28e+00  -4.43e-01    1(0)     1.8
     2   5.02e-01   1.74e+00  -3.65e-02    2(0)     2.2
     3   1.99e-01   1.69e+00  -3.02e-01    1(0) 

In [ ]:
J_F_t_2Q_2G=[]
for i in 1:100
  J_F=results[i].J_T
  push!(J_F_t_2Q_2G, J_F)
end


In [ ]:
N=100
J_F_avg_2G=0
sum=0
for i in 1:N

  sum=J_F_2Q_2G[i][end] + sum
  J_F_avg_2G= sum/N
end

In [ ]:
J_F_avg_2G

0.007416520999999999

In [ ]:
t_conv_2Q_2G=[]

for i in 1:100
  t_sec = 0
  t_conv_i = []
   for j = 1: length(iter_sec_2Q_2G[i])
      t_sec = iter_sec_2Q_2G[i][j] + t_sec
      push!(t_conv_i, t_sec)
  end
  push!(t_conv_2Q_2G, t_conv_i)
end
println(t_conv_2Q_2G)

Any[Any[8.7, 10.299999999999999, 10.7, 12.0, 12.8, 13.3, 13.700000000000001, 14.200000000000001, 14.600000000000001, 15.100000000000001, 15.500000000000002, 18.200000000000003], Any[0.8, 2.6, 4.800000000000001, 5.500000000000001, 6.600000000000001, 7.500000000000002, 8.700000000000001, 9.500000000000002, 10.200000000000001, 10.700000000000001], Any[0.4, 1.3, 1.8, 2.2, 3.3000000000000003, 4.1000000000000005, 4.9, 5.300000000000001, 5.800000000000001, 6.200000000000001, 6.600000000000001, 7.100000000000001, 7.500000000000002, 8.000000000000002, 8.400000000000002, 8.900000000000002, 9.300000000000002, 9.700000000000003, 10.200000000000003], Any[0.4, 0.9, 1.3, 2.2, 2.6, 3.1, 3.5, 4.0, 4.4, 5.0, 5.8, 6.6, 7.1, 7.5, 8.0, 8.4, 8.9, 9.3, 9.700000000000001], Any[0.5, 0.9, 2.7, 3.1, 3.6, 4.0, 4.5, 4.9, 5.4, 5.800000000000001, 6.200000000000001, 6.700000000000001, 7.200000000000001], Any[0.8, 1.5, 2.1, 2.9000000000000004, 3.4000000000000004, 3.8000000000000003, 4.300000000000001, 4.70000000000000

In [ ]:
N=100
t_conv_avg_2G=0
sum=0
for i in 1:N

  sum=t_conv_2Q_2G[i][end] + sum
  t_conv_avg_2G= sum/N
end
println(t_conv_avg_2G)

8.527999999999999


In [ ]:
N=100
t_it_avg_2G=0
sum=0
for i in 1:N
  #for j in 1:length(iter_sec_2Q_2G[i])
  sum= (t_conv_2Q_2G[i][end]/ length(iter_sec_2Q_2G[i])) +sum
  t_it_avg_2G= sum/N
  #end
end

println(t_it_avg_2G)

0.6197453142130152


In [ ]:
N=100
J_F_avg_2G=0
sum=0
for i in 1:N

  sum=J_F_2Q_2G[i][end] + sum
  J_F_avg_2G= sum/N
end
println(J_F_avg_2G)

0.007411520999999999
